[<< back to main index](../README.md)

# Multiple Linear Regression Lab : House Prices

### Overview
Estimate house prices using MLR

### Builds on
None

### Run time
approx. 20 minutes

### Notes



In [1]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()
spark

Initializing Spark...
Spark found in :  /home/ubuntu/spark
Spark config:
	 spark.app.name=TestApp
	spark.master=local[*]
	executor.memory=2g
	spark.sql.warehouse.dir=/tmp/tmpieio7jh4
	some_property=some_value
Spark UI running on port 4041



## Step 1 : House data

In [2]:
%%time

house_prices = spark.read.csv("/data/house-prices/house-sales-full.csv", \
                             header=True, inferSchema=True)
print("read {:,} records".format(house_prices.count()))


read 27,063 records
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.91 s


In [3]:
house_prices.printSchema()
house_prices.limit(5).toPandas().T

root
 |-- DocumentID: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- SalePrice: integer (nullable = true)
 |-- PropertyID: long (nullable = true)
 |-- PropertyType: string (nullable = true)
 |-- ym: string (nullable = true)
 |-- zhvi_px: integer (nullable = true)
 |-- zhvi_idx: double (nullable = true)
 |-- AdjSalePrice: decimal(8,0) (nullable = true)
 |-- NbrLivingUnits: integer (nullable = true)
 |-- SqFtLot: integer (nullable = true)
 |-- SqFtTotLiving: integer (nullable = true)
 |-- SqFtFinBasement: integer (nullable = true)
 |-- Bathrooms: double (nullable = true)
 |-- Bedrooms: integer (nullable = true)
 |-- BldgGrade: integer (nullable = true)
 |-- YrBuilt: integer (nullable = true)
 |-- YrRenovated: integer (nullable = true)
 |-- TrafficNoise: integer (nullable = true)
 |-- LandVal: integer (nullable = true)
 |-- ImpsVal: integer (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- NewConstruction: boolean (nullable = true)



,0,1,2,3,4
DocumentID,1,2,3,4,5
Date,9/16/14,6/16/06,1/29/07,2/25/08,3/29/13
SalePrice,280000,1000000,745000,425000,240000
PropertyID,1000102,1200013,1200019,2800016,2800024
PropertyType,Multiplex,Single Family,Single Family,Single Family,Single Family
ym,9/1/14,6/1/06,1/1/07,2/1/08,3/1/13
zhvi_px,405100,404400,425600,418400,351600
zhvi_idx,0.930836,0.929228,0.977941,0.961397,0.807904
AdjSalePrice,300805,1076162,761805,442065,297065
NbrLivingUnits,2,1,1,1,1


## Step 2: Create Features and Label Column in Dataframe

Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.


In [4]:
label_column = 'SalePrice'

## TODO : experiment by adding more features here
feature_columns = ["Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"]

select_columns = feature_columns + [label_column]

In [5]:
house_prices2 = house_prices.select(select_columns)
house_prices2.show()

+--------+---------+-------------+-------+---------+
|Bedrooms|Bathrooms|SqFtTotLiving|SqFtLot|SalePrice|
+--------+---------+-------------+-------+---------+
|       6|      3.0|         2400|   9373|   280000|
|       4|     3.75|         3764|  20156|  1000000|
|       4|     1.75|         2060|  26036|   745000|
|       5|     3.75|         3200|   8618|   425000|
|       4|     1.75|         1720|   8620|   240000|
|       2|      1.5|          930|   1012|   349900|
|       3|      1.5|         1750|  34465|   327500|
|       4|     1.75|         1860|  14659|   347000|
|       2|      1.0|          990|   5324|   220400|
|       4|      2.0|         1980|  10585|   437500|
|       2|      1.0|          840|  12750|   150000|
|       3|      1.0|         1750|   5200|   300000|
|       2|      1.0|          790|   5240|   292000|
|       2|      1.0|          790|   5240|   299800|
|       3|     2.25|         1530|   1245|   370000|
|       3|     2.75|         2120|   7320|   6

In [6]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
featureVector = assembler.transform(house_prices2)

featureVector = featureVector.withColumn("label", featureVector[label_column])
featureVector.show()

+--------+---------+-------------+-------+---------+--------------------+-------+
|Bedrooms|Bathrooms|SqFtTotLiving|SqFtLot|SalePrice|            features|  label|
+--------+---------+-------------+-------+---------+--------------------+-------+
|       6|      3.0|         2400|   9373|   280000|[6.0,3.0,2400.0,9...| 280000|
|       4|     3.75|         3764|  20156|  1000000|[4.0,3.75,3764.0,...|1000000|
|       4|     1.75|         2060|  26036|   745000|[4.0,1.75,2060.0,...| 745000|
|       5|     3.75|         3200|   8618|   425000|[5.0,3.75,3200.0,...| 425000|
|       4|     1.75|         1720|   8620|   240000|[4.0,1.75,1720.0,...| 240000|
|       2|      1.5|          930|   1012|   349900|[2.0,1.5,930.0,10...| 349900|
|       3|      1.5|         1750|  34465|   327500|[3.0,1.5,1750.0,3...| 327500|
|       4|     1.75|         1860|  14659|   347000|[4.0,1.75,1860.0,...| 347000|
|       2|      1.0|          990|   5324|   220400|[2.0,1.0,990.0,53...| 220400|
|       4|      

##  Step 3: Run the Linear Regression Model


In [9]:
from pyspark.ml.regression import LinearRegression

## Hint : Run model with max iterations as 10
#lr = LinearRegression(maxIter=???)
lr = LinearRegression(maxIter=50, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(featureVector)

print("Coefficents:" + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))



Coefficents:[-65211.02515143865,16273.948592039065,277.84742049805055,-0.07455647029672505]
Intercept: 106302.62644111131


**Insepct the coeffiients.  Are you seeing any ZERO (0) values?  What is the meaning of this?**

In [10]:
print("numIterations: %d" % lrModel.summary.totalIterations)
print("objectiveHistory: %s" % str(lrModel.summary.objectiveHistory))
# lrModel.summary.residuals.show()
print("RMSE: %f" % lrModel.summary.rootMeanSquaredError)

## TODO : print 'r2' from training summary
## Hint : Use TAB completion
print("r2: %f" % lrModel.summary.r2)


numIterations: 18
objectiveHistory: [0.5000000000000002, 0.4565457266170319, 0.3376986997173243, 0.30010335871879656, 0.2852977093323423, 0.27654782886180707, 0.2701349132960898, 0.26332624819112604, 0.26138384191458414, 0.25967594157902535, 0.25839392000729794, 0.2583312007398337, 0.2582630774756414, 0.2582603912713066, 0.2582596152108659, 0.258259611011856, 0.2582596096822474, 0.25825960967679096]
RMSE: 246378.173646
r2: 0.483482


#### Observe the summary output
- How many iterations did it do?
- Is it more than our 'max' iterations?
- If so, increase the 'maxIter' to 50 and run it again
- How many iterations is done now?  
- Can you explain the behavior?
- Also did the coefficients change from  maxIter=10  to maxIter=50 ?
- Did r2 change?  Why / Why-not?

## Step 4:  Predict some home prices
Create a new data frame with the following data

In [11]:
import pandas as pd

# Make a Pandas dataframe
new_data = pd.DataFrame({'Bedrooms' : [5,3,2],
                         'Bathrooms' : [3,2,1.5],
                         'SqFtTotLiving' : [4400, 1800, 1550],
                         'SqFtLot' : [10000, 5000, 4000]
             })

new_data_spark = spark.createDataFrame(new_data)
new_featureVector = assembler.transform(new_data_spark)  #run VectorAssembler again to create features.
new_featureVector.show()




+--------+---------+-------------+-------+--------------------+
|Bedrooms|Bathrooms|SqFtTotLiving|SqFtLot|            features|
+--------+---------+-------------+-------+--------------------+
|       5|      3.0|         4400|  10000|[5.0,3.0,4400.0,1...|
|       3|      2.0|         1800|   5000|[3.0,2.0,1800.0,5...|
|       2|      1.5|         1550|   4000|[2.0,1.5,1550.0,4...|
+--------+---------+-------------+-------+--------------------+



## Step 5: Run on some prediction data

Lets see the predicted output.

In [13]:
## TODO: transform dataframe with new feature vector to get predictions
## Hint : new_featureVector

predicted_prices = lrModel.transform(new_featureVector)
predicted_prices.show()

+--------+---------+-------------+-------+--------------------+------------------+
|Bedrooms|Bathrooms|SqFtTotLiving|SqFtLot|            features|        prediction|
+--------+---------+-------------+-------+--------------------+------------------+
|       5|      3.0|         4400|  10000|[5.0,3.0,4400.0,1...|1050852.4319484902|
|       3|      2.0|         1800|   5000|[3.0,2.0,1800.0,5...| 442970.0227158809|
|       2|      1.5|         1550|   4000|[2.0,1.5,1550.0,4...| 430656.7749170841|
+--------+---------+-------------+-------+--------------------+------------------+



## Step 6 : Some more FUN
Now let's feed 'SalePrice' as an input to our VectorAssembler.  
In Step-2, try this

```python
feature_columns = ["SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"]
```

Run again, and observe the following:
- look at the coefficients  (why are most of them zero?)
- look at r2

Can you explain what is going on :-) 

## Bonus Lab : Add more features
Try adding more features and see if it increases accuracy.  
Add only 'numeric' fields

**Class Competition**  
Let's see who can get the best accuracy :-) 

In [14]:
house_prices.limit(5).toPandas().T

,0,1,2,3,4
DocumentID,1,2,3,4,5
Date,9/16/14,6/16/06,1/29/07,2/25/08,3/29/13
SalePrice,280000,1000000,745000,425000,240000
PropertyID,1000102,1200013,1200019,2800016,2800024
PropertyType,Multiplex,Single Family,Single Family,Single Family,Single Family
ym,9/1/14,6/1/06,1/1/07,2/1/08,3/1/13
zhvi_px,405100,404400,425600,418400,351600
zhvi_idx,0.930836,0.929228,0.977941,0.961397,0.807904
AdjSalePrice,300805,1076162,761805,442065,297065
NbrLivingUnits,2,1,1,1,1


In [24]:
feature_columns = ["Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot", "NbrLivingUnits", "ZipCode", "SqFtFinBasement", "YrBuilt", "YrRenovated"]

select_columns = feature_columns + [label_column]
house_prices2 = house_prices.select(select_columns)
# house_prices2.show()
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
featureVector = assembler.transform(house_prices2)

featureVector = featureVector.withColumn("label", featureVector[label_column])
# featureVector.show()

lr = LinearRegression(maxIter=50, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(featureVector)

print("Coefficents:" + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

print("numIterations: %d" % lrModel.summary.totalIterations)
print("objectiveHistory: %s" % str(lrModel.summary.objectiveHistory))
# lrModel.summary.residuals.show()
print("RMSE: %f" % lrModel.summary.rootMeanSquaredError)

## TODO : print 'r2' from training summary
## Hint : Use TAB completion
print("r2: %f" % lrModel.summary.r2)

Coefficents:[-73477.99908814898,73768.21846573008,274.0636635475386,-0.09853821189760295,-61402.34671894672,0.00267023381949728,-22.620158863215813,-2276.2331180785677,7.0519584831256195]
Intercept: 4580977.927953577
numIterations: 26
objectiveHistory: [0.5000000000000002, 0.45420672157965464, 0.3357058055777957, 0.2981610722404011, 0.2816249388494852, 0.26909621699024733, 0.25841473737665577, 0.24835168915335643, 0.24646575665388196, 0.24500067642392107, 0.2441997370419274, 0.24414853402396217, 0.24410746329397429, 0.24408389068259864, 0.24408229807531476, 0.24407492122636112, 0.24407232653595223, 0.2440704250693039, 0.24407016895998418, 0.24407013053076934, 0.24407012199685646, 0.24407012176401693, 0.24407012170532452, 0.24407012169249093, 0.24407012168966283, 0.2440701216893336]
RMSE: 239514.052004
r2: 0.511862
